In [7]:
# !pip install geojson
# !pip install folium
# !pip install geopandas
# !pip install pyogrio

In [8]:
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import json
import folium
import geopandas as gpd
import pyogrio
from shapely.ops import split
from shapely.geometry import MultiLineString, LineString, Point, Polygon, MultiPolygon
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster
import json
import pandas as pd
import branca

In [ ]:
from utils_data import get_contours_city_ign, get_iris_contours_city, add_population_iris, get_vehicles_city 

In [10]:
# estimation moyennes : consommation d'un habitant qui roule en EV 
code_insee = '44055' # Identifiant qui a l'air unique pour IGN
avg_dist_hab = 30
conso_km = 0.2
avg_demand_hab = avg_dist_hab * conso_km

In [11]:
file_ign_region = '../../Data/BDCARTO/44_Loire_Atlantique/1_DONNEES_LIVRAISON_2024-12-00080/data.gpkg'
file_iris_contours = "../../data/IRIS/contours-iris.gpkg"
file_iris_population = "../../data/IRIS/base-ic-logement-2021_xlsx/base-ic-logement-2021.xlsx"

dir_ev = '../../data/EV'
file_vp = f"{dir_ev}/parc_vp_com_2011_2024.xlsx"
file_vul = f"{dir_ev}/parc_vul_com_2011_2024.xlsx"
file_pl = f"{dir_ev}/parc_pl_com2011_2024.xlsx"
file_tcp = f"{dir_ev}/parc_tcp_commune_2024.xlsx"

files_vehicules = {'vp': file_vp, 'vul': file_vul, 'pl': file_pl, 'tcp': file_tcp}
export_file_name = f"../../data/processed/df_vehicles_{code_insee}_2024.csv"

export_file_iris_pop_demand = f"../../data/processed/demand_per_iris_hab_{code_insee}.geojson"

### Contour de la Baule

In [12]:
gdf_ign_city, geometry_ign_city = get_contours_city_ign(file_ign_region, code_insee)
gdf_ign_city.head()

,code_insee,code_insee_du_departement,code_insee_de_la_region,population,surface_en_ha,code_postal,nom_officiel,geometry
266,44055,44,52,16468,2219,44500,La Baule-Escoublac,"MULTIPOLYGON (((-2.29702 47.3032, -2.2972 47.3..."


# Comparaison IRIS

In [13]:
gdf_iris = get_iris_contours_city(file_iris_contours, code_insee)
gdf_iris_population = add_population_iris(file_iris_population, gdf_iris)
pop_city = gdf_iris_population.P21_PMEN.sum() # population de la ville
df_vehicules = get_vehicles_city(code_insee, files_vehicules, export_file_name)
ratio_ev_pop = ((df_vehicules['NB_EL']+df_vehicules['NB_RECHARGEABLE'])/pop_city).iloc[0]

File already exists


In [ ]:
df_vehicules = get_vehicles_city(code_insee, files_vehicules, export_file_name)
# df_vehicules = get_vehicles_city(code_insee, files_vehicules, export_file_vehicles)
# ratio_ev_pop = ((df_vehicules['NB_EL']+df_vehicules['NB_RECHARGEABLE'])/pop_city).iloc[0]

File already exists


,Year,NB_THERMIQUE,NB_RECHARGEABLE,NB_EL


In [ ]:
if os.path.exists(export_file_name):
    print("File already exists")
    df_2024 = pd.read_csv(export_file_name)

print("Processing vehicule dataset (it may take around 10min)...")
file_vp = files_vehicules['vp'] 
file_vul = files_vehicules['vul'] 
file_pl = files_vehicules['pl'] 
file_tcp = files_vehicules['tcp'] 
print("Processing vp...")
vp = pd.read_excel(file_vp, header=3,)
vp = vp.rename(columns={'2 024': 2024})
vp_city = vp[vp['Code commune de résidence'] == code_insee]
# vp_city.to_excel(file_vp_export, index=False)
print("Processing vul...")
vul = pd.read_excel(file_vul, header=3,)
vul_city = vul[vul['Code commune de résidence'] == code_insee]
# vul_city.to_excel(file_vul_export, index=False)
print("Processing pl...")
pl = pd.read_excel(file_pl, header=3,)
pl_city = pl[pl['Code commune de résidence'] == code_insee]
# pl_city.to_excel(file_pl_export, index=False)
print("Processing tcp...")
tcp = pd.read_excel(file_tcp, header=3)
tcp_city = tcp[tcp['Code commune de résidence'] == code_insee]
# tcp_city.to_excel(file_tcp_export, index=False)
print("Merging data...")
raw_vehicules = {
    'vp': vp_city,
    'vul': vul_city, 
    'pl': pl_city,
    'tcp': tcp_city
}

Processing vehicule dataset (it may take around 10min)...
Processing vp...
Processing vul...
Processing pl...
Processing tcp...
Merging data...


NameError: name 'preprocess_vehicles_data' is not defined

In [19]:
from utils import preprocess_vehicles_data
process_vehicles = preprocess_vehicles_data(raw_vehicules)
df_grouped = process_vehicles.groupby("Year", as_index=False)[
["NB_THERMIQUE", "NB_RECHARGEABLE", "NB_EL"]
].sum()
df_2024 = df_grouped[df_grouped['Year'] == '2024']
df_2024.to_csv(export_file_name, index=False)
print("Data saved!")

  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\eliot\Documents\M2DS\Capstone\Code\utils.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["type_vhcl"] = type_vhcl
c:\Users\eliot\Documents\M2DS\Capstone\Code\utils.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["type_vhcl"] = type_vhcl
c:\Users\eliot\Documents\M2DS\Capstone\Code\utils.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

Data saved!


In [21]:

df_2024_loaded = pd.read_csv(export_file_name)
df_2024_loaded

,Year,NB_THERMIQUE,NB_RECHARGEABLE,NB_EL
0,2024,14566.64,301.585,399.273


In [15]:
gdf_iris_population.head()

,code_iris,nom_iris,geometry,P21_PMEN,ratio_appart
0,440550101,La Baule les Pins,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47...",2346.994576,0.761234
1,440550102,Centre-Benoît,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47...",1616.919056,0.766357
2,440550103,Gare-Grand Clos,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47....",1869.032071,0.655091
3,440550104,Beslon,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47...",2408.488869,0.330734
4,440550105,Escoublac,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47...",3511.317978,0.240380


In [ ]:
gdf_iris_population['demand_pop_kWh'] = gdf_iris_population['P21_PMEN'] * ratio_ev_pop * gdf_iris_population['ratio_appart'] * avg_demand_hab
gdf_iris_population.drop(columns=['P21_PMEN', 'ratio_appart'], inplace=True)
gdf_iris_population.to_file(export_file_iris_pop_demand, driver="GeoJSON")
gdf_iris_population.head()

,code_iris,nom_iris,geometry,demand_pop_kWh
0,440550101,La Baule les Pins,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47...",463.191521
1,440550102,Centre-Benoît,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47...",321.254665
2,440550103,Gare-Grand Clos,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47....",317.430572
3,440550104,Beslon,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47...",206.515939
4,440550105,Escoublac,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47...",218.825894


In [12]:
# Nb de EV dans la ville 
# note : ratio_ev * ratio_voit_poop = nb_ev / pop > juste besoin d'avoir le nb d'ev dans la ville
# nb_voitures_city = 15267 # à récupérer automatiquement
# ratio_voit_pop = nb_voitures_city / pop_city 
# ratio_ev_baule = 0.046 # 4.6% # a récupérer automatiquement 
# gdf_iris_population['demand_pop_kWh'] = gdf_iris_population['P21_PMEN'] * ratio_voit_pop * ratio_ev_baule * gdf_iris_population['ratio_appart'] * avg_demand_hab

# Plot

In [13]:
center_lat, center_lon = geometry_ign_city.centroid.y, geometry_ign_city.centroid.x
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Create a colormap based on the number of people in each carreau
min_demand = gdf_iris_population['demand_pop_kWh'].min()  # Minimum number of people
max_demand = gdf_iris_population['demand_pop_kWh'].max()  # Maximum number of people
colormap = branca.colormap.linear.YlOrRd_09.scale(min_demand, max_demand)  # Yellow to red color scale

# Add the colormap legend to the map
colormap.add_to(m)

# Iterate through each row in the GeoDataFrame and plot each feature
for _, row in gdf_iris_population.iterrows():
    # Extract geometry and properties for each feature
    feature = row['geometry']
    demand = row['demand_pop_kWh']  # Assuming this column exists
    nom = row['nom_iris']  # Assuming this column exists

    # Create a popup displaying the number of people
    text = f"{nom} : Populatio demand={demand:.2f}kWh"
    popup = folium.Popup(text, max_width=300)
    
    # Style each feature based on the number of people
    folium.GeoJson(
        feature,
        name=f"Carreau {row.name}",
        popup=popup,
        style_function=lambda x, demand=demand: {
                'fillColor': colormap(demand),  # Color based on the number of people
                'color': 'black',  # Border color
                'weight': 2,  # Border width
                'fillOpacity': 0.7  # Fill opacity
            }
    ).add_to(m)
    
# Ajouter un contrôle des couches
folium.LayerControl().add_to(m)

# Afficher la carte
m